In [1]:
from os import listdir
from os.path import isfile, join

import geopandas as gpd
from shapely.geometry import Point
import gpxpy
from tqdm import tqdm

In [7]:
folder = r'.'
tracks = list()

for file in tqdm(listdir(folder)):
    track_ = list()
    if not isfile(join(folder, file)) or file[-4:] != '.gpx':
        continue
    gpx_file = open(join(folder, file), 'r')
    gpx = gpxpy.parse(gpx_file)

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                track_.append((point.speed, Point([point.longitude, point.latitude])))
    tracks.append(track_)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:24<00:00,  3.02s/it]


In [40]:
def add_dict(init, new):
    for key, val in new.items():
        if key not in init:
            init[key] = val
        elif isinstance(init[key], list):
            init[key].append(val)
        else:
            init[key] = [init[key], val]

def mean_dict(src):
    new = src.copy()
    for key, val in new.items():
        if isinstance(val, list):
            new[key] = sum(val) / len(val)
    return new

In [38]:
from offroad_routing import Geometry

tag_values = dict()
for track in tracks:
    gdf = gpd.GeoDataFrame(track, columns=['speed', 'geometry'], crs='EPSG:4326')
    bounds = gdf.bounds
    # lon_min, lat_min, lon_max, lat_max
    bounds = bounds.minx.min(), bounds.miny.min(), bounds.maxx.max(), bounds.maxy.max()
    osm = Geometry.parse(bbox=bounds)
    speed = osm.polygons.sjoin(gdf, how='inner', predicate='contains')[['tag', 'speed']].groupby(['tag']).mean().to_dict()['speed']
    add_dict(tag_values, speed)

In [41]:
mean_dict(tag_values)

{'1': 1, '2': 2.0, '3': 1}